In [2]:
import requests
import pandas as pd
from datetime import datetime
import warnings
import numpy as np
from tqdm import tqdm
import dotenv
import os
warnings.filterwarnings('ignore')
dotenv.load_dotenv()

True

In [3]:

URL = "https://api.tequila.kiwi.com/v2/search"

In [4]:
params = {
        'fly_from': None,
        'fly_to': None,
        'date_from': '11/10/2023',
        'date_to': '17/10/2023',
        'flight_type': 'oneway',
        'curr': 'INR',
        'one_for_city': 0,
        'one_per_date': 0,
        'adults': 1,
        'children': 0,
        'selected_cabins': None,
        'only_working_days': False,
        'only_weekends': False,
        'partner_market': 'us',
        'max_stopovers': 0,
        'vehicle_type': 'aircraft',
        'limit': 25
    }

API_KEY = os.getenv("API_KEY")

In [5]:
# M: Economy,   W: Business,   C: First
seat_class_fare = {
    'M': 'economy_fare',
    'W': 'business_fare',
    'C': 'first_fare'
}
seat_class_code = {
    'economy': 'M',
    'business': 'W',
    'first': 'C'
}

In [6]:
def fetchData(origin, destination, seat_class, URL=URL, params=params):
    # M: Economy,   W: Business,   C: First
    params['fly_from'] = origin
    params['fly_to'] = destination
    params['selected_cabins'] = seat_class_code[seat_class]
    
    res = requests.get(URL, headers={"apikey":API_KEY}, params=params)
    return res.json()

In [7]:
def getdf(data):
    df = pd.DataFrame(columns=['origin','destination','depart_time','depart_weekday','duration','arrival_time','arrival_weekday','flight_no','airline_code','economy_fare','business_fare','first_fare'])
    for route in data['data']:
        d = datetime.strptime(route['route'][0]['local_departure'],"%Y-%m-%dT%H:%M:%S.%fZ")
        a = datetime.strptime(route['route'][0]['local_arrival'],"%Y-%m-%dT%H:%M:%S.%fZ")
        data_dict = {
            'origin': route['route'][0]['flyFrom'],
            'destination': route['route'][0]['flyTo'],
            'depart_time': str(d.time()),
            'depart_weekday': d.weekday(),
            'duration': str(a-d),
            'arrival_time': str(a.time()),
            'arrival_weekday': a.weekday(),
            'flight_no': f"{route['route'][0]['airline']}{route['route'][0]['flight_no']}",
            'airline_code': route['route'][0]['airline'],
            f'{seat_class_fare[route["route"][0]["fare_category"]]}': route['price']
        }
        df = pd.concat([df,pd.DataFrame(data_dict,index=[0])],ignore_index=True)


    return df

In [8]:
def getFinalDataFrame(eco_data, buss_data, first_data):
    eco_df = getdf(eco_data)
    buss_df = getdf(buss_data)
    first_df = getdf(first_data)
    
    for idx, row in buss_df.iterrows():
        match = eco_df.loc[(eco_df['origin']==row['origin']) & (eco_df['destination']==row['destination']) & (eco_df['depart_time']==row['depart_time']) & (eco_df['depart_weekday']==row['depart_weekday']) & (eco_df['arrival_time']==row['arrival_time']) & (eco_df['arrival_weekday']==row['arrival_weekday']) & (eco_df['flight_no']==row['flight_no']) & (eco_df['airline_code']==row['airline_code'])]
        for i, r in match.iterrows():
            eco_df.loc[i, 'business_fare'] = row['business_fare']
    
    for idx, row in first_df.iterrows():
        match = eco_df.loc[(eco_df['origin']==row['origin']) & (eco_df['destination']==row['destination']) & (eco_df['depart_time']==row['depart_time']) & (eco_df['depart_weekday']==row['depart_weekday']) & (eco_df['arrival_time']==row['arrival_time']) & (eco_df['arrival_weekday']==row['arrival_weekday']) & (eco_df['flight_no']==row['flight_no']) & (eco_df['airline_code']==row['airline_code'])]
        for i, r in match.iterrows():
            eco_df.loc[i, 'first_fare'] = row['first_fare']
    
    return eco_df

In [9]:
# M: Economy,   W: Business,   C: First
eco_data = fetchData('DEL', 'HND', 'economy')
buss_data = fetchData('DEL', 'HND', 'business')
first_data = fetchData('DEL', 'HND', 'first')

In [10]:
print(eco_data, buss_data, first_data, sep='\n\n')

{'search_id': 'dd9edeae-022c-33bc-6f8d-7899b44ecc4d', 'currency': 'INR', 'fx_rate': 87.423794, 'data': [{'id': '0711097b4cbd00009f9f88f0_0', 'flyFrom': 'DEL', 'flyTo': 'HND', 'cityFrom': 'New Delhi', 'cityCodeFrom': 'DEL', 'cityTo': 'Tokyo', 'cityCodeTo': 'TYO', 'countryFrom': {'code': 'IN', 'name': 'India'}, 'countryTo': {'code': 'JP', 'name': 'Japan'}, 'local_departure': '2023-10-15T18:00:00.000Z', 'utc_departure': '2023-10-15T12:30:00.000Z', 'local_arrival': '2023-10-16T05:55:00.000Z', 'utc_arrival': '2023-10-15T20:55:00.000Z', 'nightsInDest': None, 'quality': 665.54654, 'distance': 5866.18, 'duration': {'departure': 30300, 'return': 0, 'total': 30300}, 'price': 51249, 'conversion': {'EUR': 586.213405, 'INR': 51249}, 'fare': {'adults': 51249, 'children': 51249, 'infants': 51249}, 'bags_price': {'1': 0, '2': 0}, 'baglimit': {'hand_height': 40, 'hand_length': 55, 'hand_weight': 10, 'hand_width': 20, 'hold_dimensions_sum': 158, 'hold_height': 52, 'hold_length': 78, 'hold_weight': 23, '

In [11]:
# Testing New API Code
params = {
       'fly_from': 'DEL',
       'fly_to': 'HND',
       'date_from': '20/10/2023',
       'date_to': '28/10/2023',
       'flight_type': 'oneway',
       'curr': 'INR',
       'one_for_city': 0,
       'one_per_date': 0,
       'adults': 1,
       'children': 0,
       'selected_cabins': 'M',
       'only_working_days': False,
       'only_weekends': False,
       'partner_market': 'us',
       'max_stopovers': 0,
       'vehicle_type': 'aircraft',
       'limit': 25
   }

API_KEY = os.getenv("API_KEY")
URL = "https://api.tequila.kiwi.com/v2/search"

res = requests.get(URL, headers={"apikey":API_KEY}, params=params)
res.json()

{'search_id': '91f1e989-a6d3-4da7-f46e-2238a9cd082e',
 'currency': 'INR',
 'fx_rate': 87.423794,
 'data': [{'id': '0711097b4cc400003de95ed5_0',
   'flyFrom': 'DEL',
   'flyTo': 'HND',
   'cityFrom': 'New Delhi',
   'cityCodeFrom': 'DEL',
   'cityTo': 'Tokyo',
   'cityCodeTo': 'TYO',
   'countryFrom': {'code': 'IN', 'name': 'India'},
   'countryTo': {'code': 'JP', 'name': 'Japan'},
   'local_departure': '2023-10-22T18:00:00.000Z',
   'utc_departure': '2023-10-22T12:30:00.000Z',
   'local_arrival': '2023-10-23T05:55:00.000Z',
   'utc_arrival': '2023-10-22T20:55:00.000Z',
   'nightsInDest': None,
   'quality': 665.54654,
   'distance': 5866.18,
   'duration': {'departure': 30300, 'return': 0, 'total': 30300},
   'price': 51249,
   'conversion': {'EUR': 586.213405, 'INR': 51249},
   'fare': {'adults': 51249, 'children': 51249, 'infants': 51249},
   'bags_price': {'1': 0, '2': 0},
   'baglimit': {'hand_height': 40,
    'hand_length': 55,
    'hand_weight': 10,
    'hand_width': 20,
    'hol

In [12]:
df = getFinalDataFrame(eco_data, buss_data, first_data)
df

,origin,destination,depart_time,depart_weekday,duration,arrival_time,arrival_weekday,flight_no,airline_code,economy_fare,business_fare,first_fare
0,DEL,HND,18:00:00,6,11:55:00,05:55:00,0,NH838,NH,51249,NaN,NaN
1,DEL,HND,18:00:00,2,11:55:00,05:55:00,3,NH838,NH,52211,131101,214274
2,DEL,HND,19:05:00,0,11:00:00,06:05:00,1,JL30,JL,55945,NaN,NaN
3,DEL,HND,19:05:00,1,11:00:00,06:05:00,2,JL30,JL,55948,NaN,NaN
4,DEL,HND,19:05:00,3,11:00:00,06:05:00,4,JL30,JL,64404,146339,NaN
5,DEL,HND,18:00:00,1,11:55:00,05:55:00,2,NH838,NH,69398,81639,213411
6,DEL,HND,19:05:00,6,11:00:00,06:05:00,0,JL30,JL,74117,NaN,NaN
7,DEL,HND,19:05:00,2,11:00:00,06:05:00,3,JL30,JL,79196,172071,220542
8,DEL,HND,18:00:00,0,11:55:00,05:55:00,1,NH838,NH,92626,NaN,277712
9,DEL,HND,18:00:00,3,11:55:00,05:55:00,4,NH838,NH,94354,153541,NaN


In [14]:
pdf = pd.read_csv("domesticplaces.csv")
pdf.head()

,city,airport,code,country
0,Chennai,Chennai International Airport,MAA,India
1,Kolkata,Netaji Subhash Chandra Bose International Airport,CCU,India
2,Ahmedabad,Sardar Vallabhbhai Patel International Airport,AMD,India
3,Pune,Pune Airport,PNQ,India
4,Delhi,Indira Gandhi International Airport,DEL,India


In [15]:
cities = list(pdf['code'])
print(cities, len(cities))

['MAA', 'CCU', 'AMD', 'PNQ', 'DEL', 'BOM', 'BLR'] 7


In [16]:
cities[:5]

['MAA', 'CCU', 'AMD', 'PNQ', 'DEL']

In [17]:
final_df = None

In [18]:
for i in tqdm(cities):
    for j in cities:
        if i!=j:
            print(i,"->",j+": ", end='')
            eco_data = fetchData(i, j, 'economy')
            buss_data = fetchData(i, j, 'business')
            first_data = fetchData(i, j, 'first')
            df = getFinalDataFrame(eco_data, buss_data, first_data)
            if final_df is None:
                final_df = df
            else:
                # final_df = final_df.append(df, ignore_index=True).reset_index(drop=True)
                final_df = pd.concat([final_df,df],ignore_index=True)
            print("DONE")

  0%|          | 0/7 [00:00<?, ?it/s]

MAA -> CCU: DONE
MAA -> AMD: DONE
MAA -> PNQ: DONE
MAA -> DEL: DONE
MAA -> BOM: DONE
MAA -> BLR: 

 14%|█▍        | 1/7 [00:43<04:21, 43.53s/it]

DONE
CCU -> MAA: DONE
CCU -> AMD: DONE
CCU -> PNQ: DONE
CCU -> DEL: DONE
CCU -> BOM: DONE
CCU -> BLR: 

 29%|██▊       | 2/7 [01:29<03:45, 45.13s/it]

...

DONE
BOM -> MAA: DONE
BOM -> CCU: DONE
BOM -> AMD: DONE
BOM -> PNQ: DONE
BOM -> DEL: DONE
BOM -> BLR: 

 86%|████████▌ | 6/7 [05:04<00:57, 57.53s/it]

DONE
BLR -> MAA: DONE
BLR -> CCU: DONE
BLR -> AMD: DONE
BLR -> PNQ: DONE
BLR -> DEL: DONE
BLR -> BOM: 

100%|██████████| 7/7 [05:46<00:00, 49.53s/it]

DONE


In [19]:
final_df.tail()

,origin,destination,depart_time,depart_weekday,duration,arrival_time,arrival_weekday,flight_no,airline_code,economy_fare,business_fare,first_fare
985,BLR,BOM,14:20:00,2,1:30:00,15:50:00,2,I51782,I5,4438,NaN,NaN
986,BLR,BOM,14:20:00,0,1:30:00,15:50:00,0,I51782,I5,4438,NaN,NaN
987,BLR,BOM,19:00:00,1,1:35:00,20:35:00,1,UK864,UK,4464,9769,38591
988,BLR,BOM,19:00:00,0,1:35:00,20:35:00,0,UK864,UK,4464,7694,25146
989,BLR,BOM,06:00:00,3,1:30:00,07:30:00,3,SG456,SG,4667,NaN,NaN


In [20]:
final_df.to_csv("domestic_flights_short.csv")